In [1]:
import numpy as np
import pandas as pd
import scipy.io as sio
import os

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200914110624-0001
KERNEL_ID = a9f97a3a-7baf-4beb-8e88-d7efd5a82aa5


In [2]:
# original site seems to be permantly offline, using cached data
#!wget http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/97.mat
#!wget http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/98.mat
#!wget http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/99.mat
#!wget http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/100.mat
!wget https://github.com/IBM/skillsnetwork/raw/master/coursera_ai/week3/data/cwr_healthy/97.mat
!wget https://github.com/IBM/skillsnetwork/raw/master/coursera_ai/week3/data/cwr_healthy/98.mat
!wget https://github.com/IBM/skillsnetwork/raw/master/coursera_ai/week3/data/cwr_healthy/99.mat
!wget https://github.com/IBM/skillsnetwork/raw/master/coursera_ai/week3/data/cwr_healthy/100.mat


--2020-09-14 11:06:28--  https://github.com/IBM/skillsnetwork/raw/master/coursera_ai/week3/data/cwr_healthy/97.mat
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/skillsnetwork/master/coursera_ai/week3/data/cwr_healthy/97.mat [following]
--2020-09-14 11:06:29--  https://raw.githubusercontent.com/IBM/skillsnetwork/master/coursera_ai/week3/data/cwr_healthy/97.mat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3903344 (3.7M) [application/octet-stream]
Saving to: ‘97.mat’

100%[======================================>] 3,903,344   --.-K/s   in 0.1s    

2020-09-14 11:06:29 (37.7 MB/s) - ‘97.mat’ saved [3903344/3903344]

--2020-09-14 11

In [3]:
!mkdir  cwr_healthy

mkdir: cannot create directory ‘cwr_healthy’: File exists


In [4]:
!mv *.mat cwr_healthy/

In [5]:
!ls -lahr cwr_healthy/

total 34M
-rw-r--r--  1 spark 4294967294  15M Sep 14 11:06 99.mat
-rw-r--r--  1 spark 4294967294 7.4M Sep 14 11:06 98.mat
-rw-r--r--  1 spark 4294967294 3.8M Sep 14 11:06 97.mat
-rw-r--r--  1 spark 4294967294 7.5M Sep 14 11:06 100.mat
drwxr-xr-x 10 spark spark      4.0K Sep 14 11:06 ..
drwxr-xr-x  2 spark 4294967294 4.0K Sep 14 11:06 .


In [6]:
mlf=sio.loadmat('./cwr_healthy/100.mat')

In [7]:
#mlf ['X100_DE_time']
#mlf ['X100_FE_time']
mlf
#type(mlf ['X100_DE_time'])

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN, Created on: Fri Jan 28 11:25:48 2000',
 '__version__': '1.0',
 '__globals__': [],
 'X100_DE_time': array([[ 0.01460308],
        [ 0.05444862],
        [ 0.10764554],
        ...,
        [-0.02357354],
        [ 0.00521538],
        [ 0.04777292]]),
 'X100_FE_time': array([[ 0.19292182],
        [ 0.16436364],
        [ 0.09081091],
        ...,
        [ 0.10930182],
        [ 0.05218545],
        [-0.00452   ]]),
 'X100RPM': array([[1725]], dtype=uint16)}

In [8]:
def read_folder(folder):
    data = 'dummy'
    skip = False
    for file in os.listdir(folder):
        file_id = file[:-4]
        mat_file_dict = sio.loadmat(folder+file)
        del data
        for key, value in mat_file_dict.items():
            if 'DE_time' in key or 'FE_time' in key:
                a = np.array(mat_file_dict[key])
                try:
                    data
                except NameError:
                    data = a
                else:
                    if (data.shape[0] != a.shape[0]):
                        print('skipping ' + file_id)
                        skip = True
                        continue
                    data = np.hstack((data,a))
        if skip:
            skip=False
            continue
        id = np.repeat(file_id,data.shape[0])
        id.shape = (id.shape[0],1)
        data = np.hstack((id,data))
        if data.shape[1] == 2:
            zeros = np.repeat(float(0),data.shape[0])
            zeros.shape =(data.shape[0],1)
            data = np.hstack((data,zeros))
        try:
            result
        except NameError:
            result = data
        else:
            result = np.vstack((result,data))
    return result

In [9]:
result_healthy = read_folder('./cwr_healthy/')

skipping 99
skipping 99


In [10]:
result_healthy

array([['100', '0.014603076923076923', '0.19292181818181817'],
       ['100', '0.05444861538461539', '0.16436363636363635'],
       ['100', '0.10764553846153846', '0.09081090909090908'],
       ...,
       ['98', '-0.09909230769230769', '-0.007601818181818181'],
       ['98', '-0.10827138461538462', '0.0402690909090909'],
       ['98', '-0.07092923076923077', '0.06101999999999999']],
      dtype='<U32')

In [11]:
pdf = pd.DataFrame(result_healthy)

In [12]:
pdf.to_csv('result_healthy_pandas.csv', header=False, index=True)

In [13]:
# uncommented because way too much data
#!for url in `curl -s csegroups.case.edu/bearingdatacenter/pages/12k-drive-end-bearing-fault-data |grep mat |grep http |awk -F'href="' '{print $2}' |awk -F'">' '{print $1}'`; do wget $url; done
#!for url in `curl -s csegroups.case.edu/bearingdatacenter/pages/48k-drive-end-bearing-fault-data |grep mat |grep http |awk -F'href="' '{print $2}' |awk -F'">' '{print $1}'`; do wget $url; done
#!for url in `curl -s csegroups.case.edu/bearingdatacenter/pages/12k-fan-end-bearing-fault-data |grep mat |grep http |awk -F'href="' '{print $2}' |awk -F'">' '{print $1}'`; do wget $url; done
#!mkdir cwr_faulty
#!mv *.mat cwr_faulty/

In [14]:
# original site seems to be permantly offline, using cached data
# !wget http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/105.mat
# !wget http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/106.mat
# !wget http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/107.mat
# !wget http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/108.mat
!wget https://github.com/IBM/skillsnetwork/raw/master/coursera_ai/week3/data/cwr_faulty/105.mat
!wget https://github.com/IBM/skillsnetwork/raw/master/coursera_ai/week3/data/cwr_faulty/106.mat
!wget https://github.com/IBM/skillsnetwork/raw/master/coursera_ai/week3/data/cwr_faulty/107.mat
!wget https://github.com/IBM/skillsnetwork/raw/master/coursera_ai/week3/data/cwr_faulty/108.mat

--2020-09-14 11:06:51--  https://github.com/IBM/skillsnetwork/raw/master/coursera_ai/week3/data/cwr_faulty/105.mat
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/skillsnetwork/master/coursera_ai/week3/data/cwr_faulty/105.mat [following]
--2020-09-14 11:06:52--  https://raw.githubusercontent.com/IBM/skillsnetwork/master/coursera_ai/week3/data/cwr_faulty/105.mat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2910768 (2.8M) [application/octet-stream]
Saving to: ‘105.mat’

100%[======================================>] 2,910,768   --.-K/s   in 0.07s   

2020-09-14 11:06:52 (37.9 MB/s) - ‘105.mat’ saved [2910768/2910768]

--2020-09-14 

In [15]:
!mkdir cwr_faulty
!mv *.mat cwr_faulty/

mkdir: cannot create directory ‘cwr_faulty’: File exists


In [16]:
!ls cwr_faulty/

105.mat  106.mat  107.mat  108.mat


In [17]:
result_faulty = read_folder('./cwr_faulty/')

In [18]:
result_faulty

array([['105', '-0.08300435129740519', '-0.40207454545454546'],
       ['105', '-0.19573433133732535', '-0.004725454545454545'],
       ['105', '0.23341928143712576', '-0.10663090909090908'],
       ...,
       ['108', '-0.36125572854291416', '0.12265636363636362'],
       ['108', '0.3113881437125749', '0.021778181818181818'],
       ['108', '0.1705568862275449', '-0.001438181818181818']],
      dtype='<U32')

In [19]:
pdf = pd.DataFrame(result_faulty)

In [20]:
pdf.to_csv('result_faulty_pandas.csv', header=False, index=True)

In [21]:
!ls -lahr result_faulty_pandas.csv

-rw-r--r-- 1 spark 4294967294 24M Sep 14 11:07 result_faulty_pandas.csv


In [22]:
df_healthy = spark.read.csv('result_healthy_pandas.csv')
df_healthy.createOrReplaceTempView('df_healthy')

In [23]:
spark.sql('select _c1,count(_c1) as cn from df_healthy group by _c1 order by cn asc').show()

+---+------+
|_c1|    cn|
+---+------+
| 97|243938|
| 98|483903|
|100|485643|
+---+------+



In [24]:
df_faulty = spark.read.csv('result_faulty_pandas.csv')
df_faulty.createOrReplaceTempView('df_faulty')

In [25]:
spark.sql('select _c1,count(_c1) as cn from df_faulty group by _c1 order by cn asc').show()

+---+------+
|_c1|    cn|
+---+------+
|105|121265|
|106|121991|
|107|122136|
|108|122917|
+---+------+



In [26]:
#import ibmos2spark
## @hiden_cell

In [27]:
#This credentials Created by Romeo and should be replaced by mine
#credentials= {
#    'auth_url': 'https://identity.open.softlayer.com',
#    'project_id': '6aaf54352357483486ee2d4981f8ef15',
#    'region': 'dallas',
#    'user_id': 'e8a574f222e84d29b7a1987b6103fced',
#    'username': 'member_adcb54bd899a7e39d31582bccad1577f68f1992f',
#    'password': 'p*/m8,!#7s6h9poz'
#}

#configuration_name='os_d3bd5b94a9334de59a55a7fed2bedeaa_configs'
#bmos=ibmos2spark.bluemix(sc,credentials, configuration_name)

In [33]:
#bmos=ibmos2spark.bluemix(sc,credentials, bucket_name)

#from pyspark.sql import SparkSession
#spark = SparkSession.builder.getOrCreate()

In [34]:
#df_healthy.write.parquet(bmos.url('courseraai', 'cwr_healthy1.parquet'))

In [35]:
#df_faulty.write.parquet(bmos.url('courseraai', 'cwr_faulty1.parquet'))

In [31]:
# In order to obtain the correct values for "credentias", "bucket_name" and "endpoint" 
# please follow the tutorial at https://github.com/IBM/skillsnetwork/wiki/Cloud-Object-Storage-Setup
# The following Credentials created by my own

#credentials = { # your credentials go here}
credentials={
  "apikey": "eiyCrlzvyVzQloSlM9IAMMeIbBYVWEfnCtdoC9uJ5qH3",
  "cos_hmac_keys": {
    "access_key_id": "2fc45cbac4a04e89877822a08ccfd761",
    "secret_access_key": "fd876a91354f575d742141a4ec6029a024d698a934b83f42"
  },
  "endpoints": "https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints",
  "iam_apikey_description": "Auto-generated for key 2fc45cba-c4a0-4e89-8778-22a08ccfd761",
  "iam_apikey_name": "Service credentials-1",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/c781194517d84693ab87a123af03cd17::serviceid:ServiceId-9d24692b-2f05-4ec0-bd51-c7e8231a0c98",
  "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:a/c781194517d84693ab87a123af03cd17:8dba9f55-d336-4721-99b9-54d686ce6f1a::"
}


bucket_name = "cloud-object-storage-cos-standard-ub4" # your bucket name goes here
endpoint = "https://s3.eu-de.cloud-object-storage.appdomain.cloud" # your endpoint goes here

In [36]:
import base64
from ibm_botocore.client import Config
import ibm_boto3
import time




# Create client 
client = ibm_boto3.client(
    's3',
    aws_access_key_id = credentials["cos_hmac_keys"]['access_key_id'],
    aws_secret_access_key = credentials["cos_hmac_keys"]["secret_access_key"],
    endpoint_url=endpoint
)




client.upload_file('result_healthy_pandas.csv',bucket_name, 'result_healthy_pandas.csv')
client.upload_file('result_faulty_pandas.csv',bucket_name, 'result_faulty_pandas.csv')

In [38]:
df_healthy.count()

1213484

In [39]:
df_faulty.count()

488309